# 환경설정

In [1]:
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from time import sleep
from bs4 import BeautifulSoup
import re
import json
WAIT = 3
#options = webdriver.ChromeOptions()
#options.add_argument('headless')

# 함수 정의

In [2]:
# css 찾을때 까지 10초대기
def time_wait(num, code,driver):
    try:
        wait = WebDriverWait(driver, num).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, code)))
    except:
        print(code, '태그를 찾지 못하였습니다.')
        driver.quit()
    return wait

In [3]:
def find_search_input(key_word,driver):
    # 검색창 찾기
    driver.switch_to.default_content()
    time_wait(2, 'div.input_box > input.input_search',driver)
    search = driver.find_element(By.CSS_SELECTOR,'div.input_box > input.input_search')
    search.send_keys(key_word)  # 검색어 입력
    driver.implicitly_wait(3)
    search.send_keys(Keys.ENTER)  # 엔터버튼 누르기
    sleep(5)
    res = driver.page_source  # 페이지 소스 가져오기
    soup = BeautifulSoup(res, 'html.parser')  # html 파싱하여  가져온다
    sleep(1)

In [4]:
# frame 변경 메소드
def switch_frame(frame,driver):
    driver.switch_to.default_content()  # frame 초기화
    driver.switch_to.frame(frame)  # frame 변경
    driver.implicitly_wait(1)


In [5]:
# 페이지 다운
def page_down(num,driver):
    body = driver.find_element(By.CSS_SELECTOR,'body')
    body.click()
    for i in range(num):
        body.send_keys(Keys.PAGE_DOWN)

In [6]:
# 페이지 다운
def page_end(driver):
    body = driver.find_element(By.CSS_SELECTOR,'body')
    body.click()
    body.send_keys(Keys.END)

In [62]:
def find_review_btn(driver):
    sleep(2)
    try:
        menu_list = driver.find_elements(By.CSS_SELECTOR,'a.tpj9w')
        if len(menu_list) ==7:
            return menu_list[-1]
        else:
            return menu_list[-2]
    except:
        print('리뷰가 존재하지 않습니다.')

In [257]:
def crawling_menu_review(driver):
    #-----리뷰 버튼 클릭-------
    find_review_btn(driver).click()
    SCROLL_PAUSE_SEC = 1
    sleep(1)
    page_down(1,driver)
    driver.implicitly_wait(WAIT)
    #메뉴 요소 찾기
    menu_elements  = driver.find_elements(By.CSS_SELECTOR,'div.place_fixed_subtab > div > div:nth-child(1) > div > div > div > span > a > span:first-child')
    sleep(SCROLL_PAUSE_SEC)
    driver.implicitly_wait(WAIT)
    try:
        #왼쪽 버튼 찾기
        left_btn = driver.find_element(By.CSS_SELECTOR,'a.PznE8.aNw43')
        driver.implicitly_wait(WAIT)
        left_btn.click()
        driver.implicitly_wait(WAIT)
    except:
        pass

    try:
        #오른족 버튼 찾기
        right_btn = driver.find_element(By.CSS_SELECTOR, 'div.place_fixed_subtab > div > div:nth-child(1) > div > div > a.ZCqf_.aNw43')
        driver.implicitly_wait(WAIT)
    except:
        pass
    menu_list = list()
    #-----------여기까지-----------
    i = 0
    
    # menu_elentents가 존재하지 않는다면 except로 갈 것이다.
    try: 
        while True:
            if menu_elements[i].text != '': #요소가 존재하고 메뉴 텍스트도 보일때
                menu_list.append(menu_elements[i].text)
                if i == len(menu_elements)-1: # 메뉴 리스트 길이면 나온나
                    break
                i+=1
            else:
                right_btn.click()
                sleep(SCROLL_PAUSE_SEC)
                driver.implicitly_wait(WAIT)
    except:
        pass
            
    #last_scroll = driver.execute_script("return document.body.scrollHeight")
    #view_more_count = 0
    
    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        sleep(2)
        
        #if view_more_count ==19:
            #break
        try :
            #더보기 버튼 클릭
            driver.implicitly_wait(WAIT)
            driver.find_element(By.CSS_SELECTOR,'a.fvwqf').click()
            #view_more_count += 1
            
        except:
            #더보기 버튼이 없다면 while문을 빠져나온다.
            break
            # 끝까지 스크롤 다운
            
        #new_scroll = driver.execute_script("return document.body.scrollHeight")
        #driver.implicitly_wait(WAIT)
        #if last_scroll == new_scroll:
        #    break
        #else:
        #    last_scroll= new_scroll

    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    #가게 리스트 마다 방문자 리뷰 가져오기
    review_elements = driver.find_elements(By.CSS_SELECTOR,"span.zPfVt")
    review_list = []
    for i in review_elements:
        review_list.append(i.text)
    
    return menu_list, review_list

# 검색 지역 횟집의 리뷰를 크롤링.
## 한번에 하는 코드 / refresh()를 해도 응답이 없음

In [261]:
def crawling(driver, i, store_elements,review_dic):
    driver.implicitly_wait(1)
    store_elements[i].click() #li의 스토어를 클릭
    driver.implicitly_wait(WAIT) # 2초 휴식
    try:
        #다른 iframe으로 변경
        switch_frame('entryIframe',driver)
        time_wait(2, '.Fc1rA',driver)

        # -----매장명 가져오기-----
        store_name = driver.find_element(By.CSS_SELECTOR,'.Fc1rA').text
        driver.implicitly_wait(2)
        review_dic[store_name] = {}
        
        # -----평점 가져오기-----
        try :
            rank = driver.find_element(By.CSS_SELECTOR,'div.zD5Nm.f7aZ0 > div.dAsGb > span.PXMot.LXIwF > em')
            driver.implicitly_wait(WAIT)
            review_dic[store_name]['rank'] = rank.text
        except:
            pass

        # -----위치 가져오기-----
        try : 
            location = driver.find_element(By.CSS_SELECTOR, 'span.IH7VW').text
            driver.implicitly_wait(WAIT)
            review_dic[store_name]['location'] = location
        except:
            pass
        
        '''
        #---------------문제--------------
        memu_list, review_list = crawling_menu_review(driver)
        
        # 메뉴 크롤링
        review_dic[store_name]['menu'] = menu_list
        
        # 방문자 리뷰 크롤링
        review_dic[store_name]['review'] = review_list
        driver.implicitly_wait(WAIT)
        '''
    except:
        pass
    
    memu_list, review_list = crawling_menu_review(driver)
    print(len(memu_list))
    print(len(review_list))
        
    # 메뉴 크롤링
    review_dic[store_name]['menu'] = memu_list

    # 방문자 리뷰 크롤링
    review_dic[store_name]['review'] = review_list
    driver.implicitly_wait(WAIT)
    

## 3번에 걸쳐 1페이지를 크롤링

In [205]:
def review_crawling(key_word, page_num):
    review_dic = dict()
    page_num = page_num-1
    for idx in range(3):
        url = 'https://map.naver.com/v5/search'
        driver = webdriver.Chrome('./chromedriver')
        driver.get(url)
        driver.implicitly_wait(2)
        
        # 검색어 입력
        find_search_input(key_word,driver=driver)
        driver.implicitly_wait(2)
        
        #프레임 변경
        switch_frame('searchIframe',driver)
        driver.implicitly_wait(WAIT)
        
        # 페이지 버튼 찾기
        pages = driver.find_elements(By.CSS_SELECTOR,'a.mBN2s')
        if page_num <= 5:
            pages[page_num].click()
            driver.implicitly_wait(WAIT)
        else:
            driver.find_elements(By.CSS_SELECTOR,'div.zRM9F > a:nth-child(6)').click()
            driver.implicitly_wait(WAIT)
            driver.find_element(By.CSS_SELECTOR, 'div.zRM9F > a:nth-child(6)').click()
            driver.implicitly_wait(WAIT)
            
        driver.implicitly_wait(WAIT)
        page_down(40,driver)
        sleep(1)
        driver.implicitly_wait(1)
        page_down(5,driver)
        sleep(2)
        driver.execute_script("window.scrollTo(0, 0);")
        driver.implicitly_wait(WAIT)
        store_elements = driver.find_elements(By.CSS_SELECTOR,'div.ouxiq.icT4K > a:nth-child(1)')
        print('total store_list : ', len(store_elements))
        
        
        #체크포인트
        ch_pt1 = len(store_elements) // 3
        ch_pt2 = ch_pt1*2
        if idx == 0:
            for i in range(ch_pt1):
                crawling(driver, i,store_elements,review_dic)
                switch_frame('searchIframe',driver)
                if i == ch_pt1-1:
                    print('first crawling store_list', ch_pt1)
                            
        elif idx == 1:
            for i in range(ch_pt1,ch_pt2):
                crawling(driver, i,store_elements,review_dic)
                switch_frame('searchIframe',driver)
                if i == ch_pt2-1:
                    print('second crawling store_list', ch_pt2-ch_pt1)
            
        else :
            for i in range(ch_pt2,len(store_elements)):
                crawling(driver, i,store_elements,review_dic)
                switch_frame('searchIframe',driver)
                if i == len(store_elements)-1:
                    print('third crawling store_list', len(store_elements)-ch_pt2)
                

    # json 파일로 저장
    with open(f'naver_{key_word}_review_dic_page{page_num+1}.json', 'w', encoding='utf-8') as f:
        json.dump(review_dic, f, indent=4, ensure_ascii=False)
    
            
        
        
    
    
    

In [19]:
key_word_list = ['서울 횟집', '부산 횟집', '대구 횟집', '대전 횟집', '울산 횟집', '광주 횟집']

In [242]:
url = 'https://map.naver.com/v5/search'
driver = webdriver.Chrome('./chromedriver')
driver.get(url)
driver.implicitly_wait(2)

C:\Users\DanielJeong\AppData\Local\Temp\ipykernel_8816\1328651259.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('./chromedriver')


In [243]:
  # 검색어 입력
find_search_input('인천 횟집',driver=driver)
driver.implicitly_wait(2)

In [244]:
switch_frame('searchIframe',driver)
driver.implicitly_wait(WAIT)

In [166]:
page_num = 0
pages = driver.find_elements(By.CSS_SELECTOR,'a.mBN2s')
if page_num <= 5:
    pages[page_num].click()
    driver.implicitly_wait(WAIT)
else:
    driver.find_elements(By.CSS_SELECTOR,'div.zRM9F > a:nth-child(6)').click()
    driver.implicitly_wait(WAIT)
    driver.find_element(By.CSS_SELECTOR, 'div.zRM9F > a:nth-child(6)').click()
    driver.implicitly_wait(WAIT)

In [245]:
driver.implicitly_wait(WAIT)
#page_down(40,driver)
sleep(1)
driver.implicitly_wait(1)
#page_down(5,driver)
sleep(2)
driver.execute_script("window.scrollTo(0, 0);")
driver.implicitly_wait(WAIT)
store_elements = driver.find_elements(By.CSS_SELECTOR,'div.ouxiq.icT4K > a:nth-child(1)')
print('total store_list : ', len(store_elements))

total store_list :  10


In [262]:
switch_frame('searchIframe',driver)

In [267]:
review_crawling('울산 횟집',1)

C:\Users\DanielJeong\AppData\Local\Temp\ipykernel_8816\2989392167.py:6: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('./chromedriver')


total store_list :  43


WebDriverException: Message: chrome not reachable
  (Session info: chrome=104.0.5112.102)
Stacktrace:
Backtrace:
	Ordinal0 [0x006C78B3+2193587]
	Ordinal0 [0x00660681+1771137]
	Ordinal0 [0x00574070+802928]
	Ordinal0 [0x00569AB2+760498]
	Ordinal0 [0x0055CEAF+708271]
	Ordinal0 [0x00561142+725314]
	Ordinal0 [0x005655C9+742857]
	Ordinal0 [0x00575500+808192]
	Ordinal0 [0x005CD2C2+1168066]
	Ordinal0 [0x005BD5C6+1103302]
	Ordinal0 [0x005977E0+948192]
	Ordinal0 [0x005986E6+952038]
	GetHandleVerifier [0x00970CB2+2738370]
	GetHandleVerifier [0x009621B8+2678216]
	GetHandleVerifier [0x007517AA+512954]
	GetHandleVerifier [0x00750856+509030]
	Ordinal0 [0x0066743B+1799227]
	Ordinal0 [0x0066BB68+1817448]
	Ordinal0 [0x0066BC55+1817685]
	Ordinal0 [0x00675230+1856048]
	BaseThreadInitThunk [0x75A6FA29+25]
	RtlGetAppContainerNamedObjectPath [0x774075F4+228]
	RtlGetAppContainerNamedObjectPath [0x774075C4+180]


In [264]:
current_review_dic = dict()
crawling(driver, 9, store_elements,review_dic=current_review_dic)

12
31


In [265]:
current_review_dic

{'자연도횟집': {'rank': '4.26',
  'location': '인천 중구 영종해안남로 32',
  'menu': ['콘치즈',
   '치즈',
   '회무침',
   '우럭',
   '칼국수',
   '개불',
   '가리비',
   '꼬막',
   '라면',
   '산낙지',
   '매운탕',
   '자연산회'],
  'review': ['다시는 가고 싶지 않음. 손님을 대하는 태도의 기본 이 되어 있지 않음. 직원한테 콘치즈 하나더 달랬더 니 알았다고 해서 그런 줄 알았는데 주방에서 ...',
   '애기랑 우럭지리 먹으러 자주 갑니다 회무침이 맛있어요',
   '굿',
   '굿',
   '굿',
   '굿',
   '굿',
   '무심코 들린곳인데 너무 푸짐하고 맛있어요! 사장님\n인심도 너무 좋으시네요~~^^ 칼국수도 맛있었지만 \n무채김치가 어찌나 맛있던지! 영종도 오면 또 ...',
   '좋아요',
   '회 좋아요',
   '굿',
   '포장해서 1kg에 4만원 샀어요 맛도 굿굿',
   '김치가 맛있고 서비스도 좋았어요~\n감사^^',
   '갯벌 뷰에 맛난 회 만족할 만한 스끼다시',
   '맛있어요!',
   '무의도 가는길에 횟집 검색해서 갔어요. 오후4시경 이었는데, 손님은 저희밖에 없어서 전망좋은 2층 자 리에서 편하게 먹었습니다. 굴, 가리비, 산낙지...',
   '사장님친절하시고 회맛있어요',
   '아이 때문에 꽁치도 구워주시고 사장님이 친절하세요!!!',
   '맛있어요',
   '좋아요',
   '스끼다시 좋은데 넘 비싸요',
   '좋아요',
   '좋아요',
   '굳',
   '굿~',
   '그럭저럭',
   '짱맛있었어요 자연산회모듬최고',
   '서비스 굿~~^^',
   '서비스 대박입니다 ㅎㅎㅎ',
   '굿',
   '바다보면서먹기좋아요']}}

In [148]:
current_review_dic

{}

In [149]:
review_dic

{'목포회센타': {'location': '서울 관악구 남부순환로 1766'}}

In [177]:
store_elements[6].click()

In [178]:
switch_frame('entryIframe', driver)

In [179]:
memu_list, review_list = crawling_menu_review(driver)

세꼬시
매운탕
물회
전복
도다리
도다리세꼬시
광어
대구탕
생대구탕
미역국
생우럭탕
모듬회
숭어
숙성회
참치
183
친구들이랑 갔는데 회식하기 좋은 곳 같아요. 다른 텅텅 빈 자리도 많은데 굳이 옆 테이블에 아저씨들이 앉더라고요.. 그래서 진짜 시끄러웠어요. 그게...
생우럭탕 해장에 최고에요~
모임하기 좋은 장소!!
양도 푸짐 나오는음식도 많고 술이술술~~
너무너무 잘먹었습니다 회식으로도 좋아여!!!
회가 무척 맛있었습니다. 서비스는 그냥 그래요
화가 신선해요.
회 좋아하시는분이 가면 천국ㅎㅎㅎ근디 전보다 양이 좀 적어진느낌...
해산물이 신선하고 친절해요
좋아요
사장님 친절하게 서빙해주셨어요. 잘 먹었어요!
다음에 또 방문할게요^^
도다리세꼬시 숭어 광어 등 너무맛있고 친절했어요
메뉴:생대구탕
대구가 적어서 아쉽네요.
맛있게 잘 먹었어요
싱싱한 회와 소주 먹기 좋은 분위기
모임하기 좋아요
soso합니다
맛있어요
좋아요
이용했어요 !
좋아요.
굿
좋은 친구들과 즐거운 시간 보내고 맛나게 잘먹고 왔는데 생각보다 양은 적은듯 싶네요
오래만에 갔었는데…
예전 만큼 양이 많지 않았음 ㅠㅠ
좋아요
굿
깔끔하고 맛있어요!!
맛나요
적극 추천합니다
무난했습니다
굿
음식이 정갈하고 맛있습니다. 그런데 좀 시끄러워요 ㅎ
여기는 믿고 가는 숙성회 맛집입니다. 코로나로 한동
안 못 갔었는데 어제 오랜만에 친구들과의 모임에서 
옛생각도 나고 그 맛을 잃지 않고 유지하고 있...
일행이 늦었는데 따로 음식 잘 챙겨주셨어요. 음식도 맛있고, 잘 먹고 왔습니다.
거리두기가 잘 되어 있어 좋습니다
세꼬시 전문점이라면 활어 횟집 일텐데 사진에 보이 는 회는 특세꼬시모둠에 섞여 나온 것인데 한점을 먹 었는데 상태가 영 아니어서 먹지 않고 남겼습...
도다리세꼬시에 매운탕까지 완벽한 한끼엿어요ㅠㅠㅜ감사히 잘먹었습니다. 재방문의사 있습니당
오랜만에 가도 좋아요 ㅋㅋ 간만에 간건데 여전히 굿~~ㅋㄷ
굿
엄청 친절하십니다 자주 가겠습니다 맛도 좋아요
북창동직장인들점심인기메뉴 생대구탕,지리 특히여
름

In [180]:
memu_list

['세꼬시',
 '매운탕',
 '물회',
 '전복',
 '도다리',
 '도다리세꼬시',
 '광어',
 '대구탕',
 '생대구탕',
 '미역국',
 '생우럭탕',
 '모듬회',
 '숭어',
 '숙성회',
 '참치']

In [181]:
review_list

['친구들이랑 갔는데 회식하기 좋은 곳 같아요. 다른 텅텅 빈 자리도 많은데 굳이 옆 테이블에 아저씨들이 앉더라고요.. 그래서 진짜 시끄러웠어요. 그게...',
 '생우럭탕 해장에 최고에요~',
 '모임하기 좋은 장소!!',
 '양도 푸짐 나오는음식도 많고 술이술술~~',
 '너무너무 잘먹었습니다 회식으로도 좋아여!!!',
 '회가 무척 맛있었습니다. 서비스는 그냥 그래요',
 '화가 신선해요.',
 '회 좋아하시는분이 가면 천국ㅎㅎㅎ근디 전보다 양이 좀 적어진느낌...',
 '해산물이 신선하고 친절해요',
 '좋아요',
 '사장님 친절하게 서빙해주셨어요. 잘 먹었어요!',
 '다음에 또 방문할게요^^',
 '도다리세꼬시 숭어 광어 등 너무맛있고 친절했어요',
 '메뉴:생대구탕\n대구가 적어서 아쉽네요.',
 '맛있게 잘 먹었어요',
 '싱싱한 회와 소주 먹기 좋은 분위기\n모임하기 좋아요',
 'soso합니다',
 '맛있어요',
 '좋아요',
 '이용했어요 !',
 '좋아요.',
 '굿',
 '좋은 친구들과 즐거운 시간 보내고 맛나게 잘먹고 왔는데 생각보다 양은 적은듯 싶네요',
 '오래만에 갔었는데…\n예전 만큼 양이 많지 않았음 ㅠㅠ',
 '좋아요',
 '굿',
 '깔끔하고 맛있어요!!',
 '맛나요',
 '적극 추천합니다',
 '무난했습니다',
 '굿',
 '음식이 정갈하고 맛있습니다. 그런데 좀 시끄러워요 ㅎ',
 '여기는 믿고 가는 숙성회 맛집입니다. 코로나로 한동\n안 못 갔었는데 어제 오랜만에 친구들과의 모임에서 \n옛생각도 나고 그 맛을 잃지 않고 유지하고 있...',
 '일행이 늦었는데 따로 음식 잘 챙겨주셨어요. 음식도 맛있고, 잘 먹고 왔습니다.',
 '거리두기가 잘 되어 있어 좋습니다',
 '세꼬시 전문점이라면 활어 횟집 일텐데 사진에 보이 는 회는 특세꼬시모둠에 섞여 나온 것인데 한점을 먹 었는데 상태가 영 아니어서 먹지 않고 남겼습...',
 '도다리세꼬시에 매운탕까지 완벽한 한끼엿어요ㅠㅠㅜ감사히 잘먹었습니다. 재방문의사 있습니당',

In [156]:
menu_elements  = driver.find_elements(By.CSS_SELECTOR,'div.place_fixed_subtab > div > div:nth-child(1) > div > div > div > span > a > span:first-child')

In [160]:
menu_elements[13].text

'고기'

In [162]:
del menu_elements

In [193]:
def ex1():
    seminar = {'정성원': 'SSS'}
    ex2(seminar)
    return seminar

In [234]:
def ex2(seminar):
    seminar['김동현'] = 'SS'
    seminar['탄비르'] = ex3()

In [235]:
def ex3():
    return 'SS'

In [236]:
print(ex1())

{'정성원': 'SSS', '김동현': 'SS', '탄비르': 'SS'}


In [201]:
del review_dic